In [ ]:
import cv2

print(f"OpenCV version is: {cv2.__version__}")

if True:
    # use cv2_imshow from matplotlib (eg. Vscode)
    import matplotlib.pyplot as plt

    def cv2_imshow(img):
        plt.figure(figsize=(10,10))
        plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        plt.axis("off")
        plt.show()

In [ ]:
# TODO: detect, draw and show markers on images 1-6.png
import os

def get_all_file_paths(folder):
  file_paths = []

  for dirpath, _, filenames in os.walk(folder):
    for filename in filenames:
      if dirpath == folder:
        full_path = os.path.join(dirpath, filename)
        file_paths.append(full_path)

  return file_paths

folder_path = 'data'
all_files = get_all_file_paths(folder_path)

dictionary = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_APRILTAG_16h5)
parameters = cv2.aruco.DetectorParameters()
detector = cv2.aruco.ArucoDetector(dictionary, parameters)

for path in all_files:
   print(path)

In [67]:
chessboard_shape = (8,5)

def detect_chessboard(filename):
  current = cv2.imread(filename)
  gray = cv2.cvtColor(current, cv2.COLOR_BGR2GRAY)
  ret, corners = cv2.findChessboardCorners(gray, (8,5), None)

  if ret:
    corners2 = cv2.cornerSubPix(
      gray,
      corners,
      (11,11),
      (-1,-1),
      (cv2.TermCriteria_MAX_ITER + cv2.TERM_CRITERIA_EPS, 30, 0.0001)
    )

    cv2.drawChessboardCorners(current, (8,5), corners2, ret)
    #cv2_imshow(current)
    return corners2
  else:
    print("Couldn't detect corners.")
    return None

detect_chessboard('data/img01.jpg')
""

''

In [68]:
import numpy as np

def get_objpoints(shape, width):
  x, y = shape
  xx, yy = np.meshgrid( 
    np.linspace(0, width * (x - 1), x), 
    np.linspace(0, width * (y - 1), y), 
  )
  xx, yy = xx.reshape(-1), yy.reshape(-1)
  objarr = np.vstack([xx, yy, np.zeros(len(xx))]).T

  return objarr.astype(np.float32)

def g_op2(shape, width):
  x, y = shape
  objp = np.zeros((x*y,3), np.float32)
  objp[:,:2] = np.mgrid[0:x,0:y].T.reshape(-1,2)
  return (objp * width).astype(np.float32)

get_objpoints(chessboard_shape, 30).shape[0]
#g_op2((4,3), 30)

40

In [69]:
def calibration(filename):
  current = cv2.imread(filename)
  gray = cv2.cvtColor(current, cv2.COLOR_BGR2GRAY)

  objpoints = get_objpoints(chessboard_shape, 30)
  corners = detect_chessboard(filename)
  
  # print(objpoints)
  # print(corners.shape)
  # print(objpoints.shape)
  # objpoints = np.zeros((40, 3)).astype(np.float32)
  # print(objpoints)

  return cv2.calibrateCamera(
    objectPoints = [objpoints], 
    imagePoints = [corners],
    imageSize = gray.shape[:2],
    cameraMatrix = None,
    distCoeffs = None,
  )

for v in calibration('data/img02.jpg'):
  print(f"{v}\n\n")

0.18883259302893984


[[1.04551836e+03 0.00000000e+00 7.18986213e+02]
 [0.00000000e+00 1.13760876e+03 7.82422844e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]


[[-0.20996711 -0.00698892 -0.06636794 -0.00110452 -0.01029725]]


(array([[-0.56434907],
       [ 0.07345001],
       [ 0.56374931]]),)


(array([[-123.13634679],
       [-254.04036664],
       [ 457.72730112]]),)




In [ ]:
calibration_res = calibration('data/img02.jpg')
ret, cameraMatrix, distCoeffs, rvecs, tvecs = calibration_res

#detect_chessboard(all_files[0])

#for file in all_files:
#  detect_draw_show(file)

(array([[-0.56434907],
        [ 0.07345001],
        [ 0.56374931]]),)